In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("spark://cm1:7077").appName("teste kafka").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/07 19:14:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
lines = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "cm4:9092") \
    .option("subscribe", "teste") \
    .option('includeTimestamp', 'true') \
    .load()

In [4]:
from pyspark.sql.functions import explode, split, col

In [5]:
words = lines.select(
    explode(split(col("value"), "\s+")).alias("word"),
    lines.timestamp
)

In [6]:
words.writeStream \
    .format('console') \
    .option('truncate', 'false') \
    .start()

23/02/05 19:51:23 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e7f0bdf1-cb15-4d13-addc-36a07bbf8b7e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/02/05 19:51:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+---------+
|word|timestamp|
+----+---------+
+----+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----------------------+
|word    |timestamp              |
+--------+-----------------------+
|testando|2023-02-05 19:51:29.465|
|1       |2023-02-05 19:51:29.465|
|2       |2023-02-05 19:51:29.465|
+--------+-----------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----------------------+
|word |timestamp              |
+-----+-----------------------+
|ola  |2023-02-05 19:51:32.174|
|mundo|2023-02-05 19:51:32.174|
+-----+-----------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+----+-----------------------+
|word|timestamp              |
+----+-----------------------+
|tudo|2023-02-05 19:51:38.594|
|bem |2023-02-05 19:51:38.594|
|?   |2023-02-05 19:51:38.594|
+----+-----------------------+



In [7]:
spark.stop()